# We will see how to use PIVOT function for dataframe
- you can download the data from https://datamarket.com/data/set/22ky/monthly-total-number-of-pigs-slaughtered-in-victoria-jan-1980-august-1995#

In [1]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql
import org.apache.spark.sql.types._
import org.apache.spark.sql.Encoders
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.functions._

In [2]:
val conf = new SparkConf().setAppName("Flight status")
val sc = new SparkContext(conf)
val spark = new SQLContext(sc) 

conf = org.apache.spark.SparkConf@1e55a2b7
sc = org.apache.spark.SparkContext@50835744
spark = org.apache.spark.sql.SQLContext@5c275e9


org.apache.spark.sql.SQLContext@5c275e9

In [3]:
val data = spark.read.format("csv")
              .option("header","true")
              .option("delimiter", ",")
              .option("inferSchema", "true")
              .load("/user/viswatejaster9073/ML/monthly-total-number-of-pigs-sla.csv").withColumnRenamed("Monthly total number of pigs slaughtered in Victoria. Jan 1980 ? August 1995","count")

data = [Month: string, count: int]


[Month: string, count: int]

In [24]:
data.printSchema
data.show()

root
 |-- Month: string (nullable = true)
 |-- count: integer (nullable = true)

+-------+------+
|  Month| count|
+-------+------+
|1980-01| 76378|
|1980-02| 71947|
|1980-03| 33873|
|1980-04| 96428|
|1980-05|105084|
|1980-06| 95741|
|1980-07|110647|
|1980-08|100331|
|1980-09| 94133|
|1980-10|103055|
|1980-11| 90595|
|1980-12|101457|
|1981-01| 76889|
|1981-02| 81291|
|1981-03| 91643|
|1981-04| 96228|
|1981-05|102736|
|1981-06|100264|
|1981-07|103491|
|1981-08| 97027|
+-------+------+
only showing top 20 rows



# I want to extract Month and year in to two diffrent columns

In [4]:
data.createOrReplaceTempView("data")

In [5]:
val data_month_year = spark.sql("select year(to_date(Month)) as year,month(to_date(Month)) as month,count from data")

data_month_year = [year: int, month: int ... 1 more field]


[year: int, month: int ... 1 more field]

In [32]:
data_month_year.show(5)

+----+-----+------+
|year|month| count|
+----+-----+------+
|1980|    1| 76378|
|1980|    2| 71947|
|1980|    3| 33873|
|1980|    4| 96428|
|1980|    5|105084|
+----+-----+------+
only showing top 5 rows



# Now we are using PIVOT function on month column 

In [17]:
val s =data_month_year
  .groupBy($"year")
  .pivot("month").sum("count").orderBy($"year".asc).withColumnRenamed("1" ,"JAN").withColumnRenamed("2" ,"FEB")
.withColumnRenamed("3" ,"MAR").withColumnRenamed("4" ,"APR").withColumnRenamed("5" ,"MAY").withColumnRenamed("6" ,"JUN")
.withColumnRenamed("7" ,"JUL").withColumnRenamed("8" ,"AUG").withColumnRenamed("9" ,"SEP").withColumnRenamed("10" ,"OCT")
.withColumnRenamed("11" ,"NOV").withColumnRenamed("12" ,"DEC")

s = [year: int, null: bigint ... 12 more fields]


[year: int, null: bigint ... 12 more fields]

### If you see the below response we make rows(months) in to columns 

In [31]:
s.show()

+----+----+------+------+------+------+------+------+------+------+------+------+------+------+
|year|null|   JAN|   FEB|   MAR|   APR|   MAY|   JUN|   JUL|   AUG|   SEP|   OCT|   NOV|   DEC|
+----+----+------+------+------+------+------+------+------+------+------+------+------+------+
|null|null|  null|  null|  null|  null|  null|  null|  null|  null|  null|  null|  null|  null|
|1980|null| 76378| 71947| 33873| 96428|105084| 95741|110647|100331| 94133|103055| 90595|101457|
|1981|null| 76889| 81291| 91643| 96228|102736|100264|103491| 97027| 95240| 91680|101259|109564|
|1982|null| 76892| 85773| 95210| 93771| 98202| 97906|100306| 94089|102680| 77919| 93561|117062|
|1983|null| 81225| 88357|106175| 91922|104114|109959| 97880|105386| 96479| 97580|109490|110191|
|1984|null| 90974| 98981|107188| 94177|115097|113696|114532|120110| 93607|110925|103312|120184|
|1985|null|103069|103351|111331|106161|111590| 99447|101987| 85333| 86970|100561| 89543| 89265|
|1986|null| 82719| 79498| 74846| 73819| 

## we are deleting the column name with null

In [18]:
val columns = s.columns.filter(_!="null")

columns = Array(year, JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, NOV, DEC)


[year, JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, NOV, DEC]

In [19]:
val final_data = s.selectExpr(columns:_*)

final_data = [year: int, JAN: bigint ... 11 more fields]


[year: int, JAN: bigint ... 11 more fields]

In [23]:
final_data.show()

+----+------+------+------+------+------+------+------+------+------+------+------+------+
|year|   JAN|   FEB|   MAR|   APR|   MAY|   JUN|   JUL|   AUG|   SEP|   OCT|   NOV|   DEC|
+----+------+------+------+------+------+------+------+------+------+------+------+------+
|null|  null|  null|  null|  null|  null|  null|  null|  null|  null|  null|  null|  null|
|1980| 76378| 71947| 33873| 96428|105084| 95741|110647|100331| 94133|103055| 90595|101457|
|1981| 76889| 81291| 91643| 96228|102736|100264|103491| 97027| 95240| 91680|101259|109564|
|1982| 76892| 85773| 95210| 93771| 98202| 97906|100306| 94089|102680| 77919| 93561|117062|
|1983| 81225| 88357|106175| 91922|104114|109959| 97880|105386| 96479| 97580|109490|110191|
|1984| 90974| 98981|107188| 94177|115097|113696|114532|120110| 93607|110925|103312|120184|
|1985|103069|103351|111331|106161|111590| 99447|101987| 85333| 86970|100561| 89543| 89265|
|1986| 82719| 79498| 74846| 73819| 77029| 78446| 86978| 75878| 69571| 75722| 64182| 77357|

# Lets sum the slughts for all months for each year 

In [24]:
val columnsToSum = List(col("JAN"), col("FEB"), col("MAR"), col("APR"), col("MAY"),col("JUN"),col("JUL"),col("AUG"),
                        col("SEP"),col("OCT"),col("NOV"),col("DEC"))

columnsToSum = List(JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, NOV, DEC)


List(JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, NOV, DEC)

In [29]:
final_data.withColumn("Total_slut_In_a_Year",columnsToSum.reduce(_+_)).show()

+----+------+------+------+------+------+------+------+------+------+------+------+------+--------------------+
|year|   JAN|   FEB|   MAR|   APR|   MAY|   JUN|   JUL|   AUG|   SEP|   OCT|   NOV|   DEC|Total_slut_In_a_Year|
+----+------+------+------+------+------+------+------+------+------+------+------+------+--------------------+
|null|  null|  null|  null|  null|  null|  null|  null|  null|  null|  null|  null|  null|                null|
|1980| 76378| 71947| 33873| 96428|105084| 95741|110647|100331| 94133|103055| 90595|101457|             1079669|
|1981| 76889| 81291| 91643| 96228|102736|100264|103491| 97027| 95240| 91680|101259|109564|             1147312|
|1982| 76892| 85773| 95210| 93771| 98202| 97906|100306| 94089|102680| 77919| 93561|117062|             1133371|
|1983| 81225| 88357|106175| 91922|104114|109959| 97880|105386| 96479| 97580|109490|110191|             1198758|
|1984| 90974| 98981|107188| 94177|115097|113696|114532|120110| 93607|110925|103312|120184|             1